# 4.8 Grouping Data & Aggregating Variables 

## This script contains the following: 
1. Importing libraries and data 
2. Find aggregated mean of order_number column grouped by department_id 
3.Creating loyalty flags for existing customers using the transform () and loc() 
4. Creating a spending flag for each user based on average price 
5. Creating an order frequency flag. 


### Importing libraries and data 

In [1]:
#Import libraries 
import pandas as pd
import numpy as np
import os

In [2]:
#Set Path 
path= r'C:\Users\hmaio\Documents\4.0 Instacart Project'

In [3]:
# Import Data 
df_ords_prods_merged = pd.read_pickle(os.path.join(path, '4.0 Data', 'Prepared Data', 'orders_products_adjusted.pkl'))

### Find aggregated mean of order_number grouped by department_id

In [4]:
# spliting data into groups based on department_id and apply agg() function to each group to obtain mean values 
df_ords_prods_merged.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1,16.559358
2,18.413176
3,18.279600
4,18.915890
5,16.497751
6,17.609390
7,18.303975
8,16.383301


Comparing this output to the ones from the subset there a few differences. Department 4 average is still 19 however it is no longer the most. That belongs to department 21. Department 11 average is closer to other departments now. FInally department 8 has the lowest average of 16.3.

### Creating loyalty flags for existing customers 

In [5]:
#creating max order column 
df_ords_prods_merged['max_order'] =df_ords_prods_merged.groupby(['user_id'])['order_number'].transform(np.max)

In [6]:
#checking output 
df_ords_prods_merged.head(15)

,order_id,user_id,order_number,orders_day_of_week,order_time_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge,price_range_loc,busiest_days,busiest_period_of_day,max_order
0,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,both,Mid-Range Product,Least busiest days,Average orders,10
1,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0,both,Mid-Range Product,Least busiest days,Most orders,10
2,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0,both,Mid-Range Product,Least busiest days,Average orders,10
3,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,9.0,both,Mid-Range Product,Least busiest days,Most orders,10
4,3367565,1,6,2,7,19.0,196,1,1,Soda,77,7,9.0,both,Mid-Range Product,Regularly busy,Average orders,10
5,550135,1,7,1,9,20.0,196,1,1,Soda,77,7,9.0,both,Mid-Range Product,Busiest days,Average orders,10
6,3108588,1,8,1,14,14.0,196,2,1,Soda,77,7,9.0,both,Mid-Range Product,Busiest days,Most orders,10
7,2295261,1,9,1,16,0.0,196,4,1,Soda,77,7,9.0,both,Mid-Range Product,Busiest days,Most orders,10
8,2550362,1,10,4,8,30.0,196,1,1,Soda,77,7,9.0,both,Mid-Range Product,Least busiest days,Average orders,10
9,2968173,15,15,1,9,7.0,196,2,0,Soda,77,7,9.0,both,Mid-Range Product,Busiest days,Average orders,22


In [7]:
#creating loyal flag 
df_ords_prods_merged.loc[df_ords_prods_merged['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'
df_ords_prods_merged.loc[(df_ords_prods_merged['max_order'] <= 40) & (df_ords_prods_merged['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'
df_ords_prods_merged.loc[df_ords_prods_merged['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [8]:
# frequency check 
df_ords_prods_merged['loyalty_flag'].value_counts (dropna= False)

Regular customer    15081691
Loyal customer      10095381
New customer         5151691
Name: loyalty_flag, dtype: int64

In [9]:
#check the basic statistics of the product prices for each loyal customer  
df_ords_prods_merged.groupby('loyalty_flag').agg({'prices':['mean','max','min']})


prices              
                       mean      max  min
loyalty_flag                             
Loyal customer    10.402162  99999.0  1.0
New customer      13.396333  99999.0  1.0
Regular customer  12.546842  99999.0  1.0

New customers are spending the most while loyal customers spend the least

###  Creating a spending flag

In [10]:
#creating average spent column 
df_ords_prods_merged['avg_spent'] =df_ords_prods_merged.groupby(['user_id'])['prices'].transform(np.mean)

In [11]:
#check output 
df_ords_prods_merged.head()

,order_id,user_id,order_number,orders_day_of_week,order_time_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge,price_range_loc,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_spent
0,2398795,1,2,3,7,15.0,196,1,1,Soda,77,7,9.0,both,Mid-Range Product,Least busiest days,Average orders,10,New customer,6.372222
1,473747,1,3,3,12,21.0,196,1,1,Soda,77,7,9.0,both,Mid-Range Product,Least busiest days,Most orders,10,New customer,6.372222
2,2254736,1,4,4,7,29.0,196,1,1,Soda,77,7,9.0,both,Mid-Range Product,Least busiest days,Average orders,10,New customer,6.372222
3,431534,1,5,4,15,28.0,196,1,1,Soda,77,7,9.0,both,Mid-Range Product,Least busiest days,Most orders,10,New customer,6.372222
4,3367565,1,6,2,7,19.0,196,1,1,Soda,77,7,9.0,both,Mid-Range Product,Regularly busy,Average orders,10,New customer,6.372222


In [12]:
# creating spending flag 
df_ords_prods_merged.loc[df_ords_prods_merged['avg_spent'] >= 10, 'spender_flag'] = 'High spender'
df_ords_prods_merged.loc[df_ords_prods_merged['avg_spent'] < 10, 'spender_flag'] = 'Low spender'

In [13]:
# check frequency 
df_ords_prods_merged['spender_flag'].value_counts (dropna= False)

Low spender     29730505
High spender      598258
Name: spender_flag, dtype: int64

###  Order Frequency Flag 

In [14]:
# creating median days since order column
df_ords_prods_merged['median_days_since_order'] =df_ords_prods_merged.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

In [15]:
#check output 
df_ords_prods_merged.head()

,order_id,user_id,order_number,orders_day_of_week,order_time_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,prices,_merge,price_range_loc,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_spent,spender_flag,median_days_since_order
0,2398795,1,2,3,7,15.0,196,1,1,Soda,...,9.0,both,Mid-Range Product,Least busiest days,Average orders,10,New customer,6.372222,Low spender,20.5
1,473747,1,3,3,12,21.0,196,1,1,Soda,...,9.0,both,Mid-Range Product,Least busiest days,Most orders,10,New customer,6.372222,Low spender,20.5
2,2254736,1,4,4,7,29.0,196,1,1,Soda,...,9.0,both,Mid-Range Product,Least busiest days,Average orders,10,New customer,6.372222,Low spender,20.5
3,431534,1,5,4,15,28.0,196,1,1,Soda,...,9.0,both,Mid-Range Product,Least busiest days,Most orders,10,New customer,6.372222,Low spender,20.5
4,3367565,1,6,2,7,19.0,196,1,1,Soda,...,9.0,both,Mid-Range Product,Regularly busy,Average orders,10,New customer,6.372222,Low spender,20.5


In [16]:
#creating frequency flag 
df_ords_prods_merged.loc[df_ords_prods_merged['median_days_since_order'] > 20, 'frequency_flag'] = 'Non-frequent customer'
df_ords_prods_merged.loc[(df_ords_prods_merged['median_days_since_order'] > 10) & (df_ords_prods_merged['median_days_since_order'] <= 20), 'frequency_flag'] = 'Regular customer'
df_ords_prods_merged.loc[df_ords_prods_merged['median_days_since_order'] < 10, 'frequency_flag'] = 'Frequent customer'

In [17]:
#check frequency 
df_ords_prods_merged['frequency_flag'].value_counts (dropna = False)

Frequent customer        19483250
Regular customer          6594542
Non-frequent customer     3058931
NaN                       1192040
Name: frequency_flag, dtype: int64

In [18]:
# Exporting data 
df_ords_prods_merged.to_pickle(os.path.join(path, '4.0 Data','Prepared Data', 'orders_products_updated.pkl'))

### To replace outliners with missing value 

In [ ]:
# this code (not run) would change all the prices above 100 in the price column to nan. 
ords_prods_merge.loc[ords_prods_merge['prices'] >100, 'prices'] = np.nan